Objetivo: Usando el dataset winequality-red.csv, el cual consiste en datos de vinos rojos basados en datos físico-químicos y una métrica de calidad de vino. Predecir la calidad del vino usando los atributos físico-químicos del mismo.

In [1]:
# Importa las librerias necesarias
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from auxiliary import plot_boundary, plot_svm_margins

In [3]:
# Carga la ruta al archivo de datos
ruta = os.path.dirname((os.path.abspath('models')))
ruta_datos = os.path.join(ruta, "dataset/winequality-red_procesados.csv")

# Lectura del archivo a DataFrame
wines = pd.read_csv(ruta_datos)

Consideraciones: Como el target 'quality' es de naturaleza numérica (int64), no conviene utilizar un modelo SVM de clasificación multiclase (serían 11 clases, enteros de 0 a 10). Con lo cual elige la regresión.

In [6]:
# Importa las clases necesarias
from sklearn.model_selection import train_test_split

In [7]:
# Separa las caracteristicas predictoras de la variable a predecir.
X = wines.loc[:, ["fixed acidity", "volatile acidity", "residual sugar", "chlorides", "total sulfur dioxide", "sulphates", "alcohol"]].values
y = wines.loc[:, "quality"].values

In [8]:
# Separa los sets de entrenamoento y de prueba en 80/20%.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0, stratify=y)

In [9]:
# Estandariza las caracteristicas
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train) 
X_test = sc_X.transform(X_test)

Consideraciones: Como metrica se usa MAE, dado que estamos en un caso de regresión.

In [10]:
# Busqueda de hiperparametros por grilla
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

svm_wines = SVR()

grid = GridSearchCV(svm_wines,
                    [{"C": [0.01, 0.1, 1, 5, 10, 100], "kernel": ["linear"]},
                     {"C": [0.01, 0.1, 1, 5, 10, 100], "gamma": [0.1, 0.5, 1, 2, 10, 100], "kernel": ["rbf", "sigmoid"]},
                     {"C": [0.01, 0.1, 1, 5, 10, 100], "degree": [2, 3, 4, 5, 6], "kernel": ["poly"]}],
                    refit=True,
                    cv=5,
                    scoring='neg_mean_absolute_error') # MAE es mejor en el caso de regresion
grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.01, 0.1, 1, 5, 10, 100],
                          'kernel': ['linear']},
                         {'C': [0.01, 0.1, 1, 5, 10, 100],
                          'gamma': [0.1, 0.5, 1, 2, 10, 100],
                          'kernel': ['rbf', 'sigmoid']},
                         {'C': [0.01, 0.1, 1, 5, 10, 100],
                          'degree': [2, 3, 4, 5, 6], 'kernel': ['poly']}],
             scoring='neg_mean_absolute_error')

In [11]:
grid.best_params_

{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}

In [12]:
# Se evalua con el set de testeo
from sklearn.metrics import (mean_absolute_error, r2_score,
                             root_mean_squared_error, 
                             mean_absolute_percentage_error)

svm_hitters_best = grid.best_estimator_

y_pred = svm_hitters_best.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE de testeo fue: {mae}")
print(f"RMSE de testeo fue: {rmse}")
print(f"MAPE de testeo fue: {mape}")
print(f"R2 de testeo fue: {r2}")

MAE de testeo fue: 0.48995437289565086
RMSE de testeo fue: 0.6585483338311917
MAPE de testeo fue: 0.08790952249576117
R2 de testeo fue: 0.30860251304737496


Conclusión: el modelo es bastante malo.

Bibliografia

Cortez,Paulo, Cerdeira,A., Almeida,F., Matos,T., and Reis,J.. (2009). Wine Quality. UCI Machine Learning Repository. https://doi.org/10.24432/C56S3T.